In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
%matplotlib inline

In [35]:
initial_data =pd.read_csv('initial_data.csv')

In [36]:
initial_data.shape

(3319256, 12)

In [37]:
initial_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3319256 entries, 0 to 3319255
Data columns (total 12 columns):
JOBID        object
ACCOUNT      object
USER         object
REQMEM       float64
USEDMEM      float64
REQTIME      int64
USEDTIME     int64
NODES        int64
CPUS         int64
PARTITION    object
EXITCODE     object
SUBMIT       object
dtypes: float64(2), int64(4), object(6)
memory usage: 303.9+ MB


In [38]:
initial_data.head()

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,SUBMIT
0,6794220,laxtons,mikala,15488.0,7201.07,1209600,955970,1,16,production,0:0,2019-03-09T16:38:44
1,6983193_1880,discovery,serina,12288.0,6516.10,1188000,1070801,1,1,production,0:0,2019-03-07T18:39:59
2,7055502_1863,discovery,serina,12288.0,0.09,1188000,1153656,1,1,production,0:0,2019-03-11T10:06:06
3,7055502_1800,discovery,serina,12288.0,8333.82,1188000,1050659,1,1,production,0:0,2019-03-11T10:06:06
4,7055502_1806,discovery,serina,12288.0,8304.03,1188000,981826,1,1,production,0:0,2019-03-11T10:06:06


In [39]:
%%time 
def array_jobs(job):
    array_split=job.split('_')
    if len(array_split)==2:
        jobs=job#array_split
    else:
        jobs=0
        
    return jobs

Wall time: 0 ns


In [40]:
initial_data.JOBID=initial_data.JOBID.apply(array_jobs)

In [41]:
initial_data.head()

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,SUBMIT
0,0,laxtons,mikala,15488.0,7201.07,1209600,955970,1,16,production,0:0,2019-03-09T16:38:44
1,6983193_1880,discovery,serina,12288.0,6516.10,1188000,1070801,1,1,production,0:0,2019-03-07T18:39:59
2,7055502_1863,discovery,serina,12288.0,0.09,1188000,1153656,1,1,production,0:0,2019-03-11T10:06:06
3,7055502_1800,discovery,serina,12288.0,8333.82,1188000,1050659,1,1,production,0:0,2019-03-11T10:06:06
4,7055502_1806,discovery,serina,12288.0,8304.03,1188000,981826,1,1,production,0:0,2019-03-11T10:06:06


In [42]:
initial_data.drop(initial_data[initial_data['JOBID']==0].index,inplace=True)

In [43]:
initial_data.head()

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,SUBMIT
1,6983193_1880,discovery,serina,12288.0,6516.10,1188000,1070801,1,1,production,0:0,2019-03-07T18:39:59
2,7055502_1863,discovery,serina,12288.0,0.09,1188000,1153656,1,1,production,0:0,2019-03-11T10:06:06
3,7055502_1800,discovery,serina,12288.0,8333.82,1188000,1050659,1,1,production,0:0,2019-03-11T10:06:06
4,7055502_1806,discovery,serina,12288.0,8304.03,1188000,981826,1,1,production,0:0,2019-03-11T10:06:06
5,7055502_1823,discovery,serina,12288.0,7693.11,1188000,789220,1,1,production,0:0,2019-03-11T10:06:06


In [44]:
initial_data['EXITCODE'].value_counts()

0:0    2575548
Name: EXITCODE, dtype: int64

In [45]:
initial_data[['number','sub_number']]=initial_data.JOBID.str.split('_',expand=True)

In [46]:
initial_data.head()

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,SUBMIT,number,sub_number
1,6983193_1880,discovery,serina,12288.0,6516.10,1188000,1070801,1,1,production,0:0,2019-03-07T18:39:59,6983193,1880
2,7055502_1863,discovery,serina,12288.0,0.09,1188000,1153656,1,1,production,0:0,2019-03-11T10:06:06,7055502,1863
3,7055502_1800,discovery,serina,12288.0,8333.82,1188000,1050659,1,1,production,0:0,2019-03-11T10:06:06,7055502,1800
4,7055502_1806,discovery,serina,12288.0,8304.03,1188000,981826,1,1,production,0:0,2019-03-11T10:06:06,7055502,1806
5,7055502_1823,discovery,serina,12288.0,7693.11,1188000,789220,1,1,production,0:0,2019-03-11T10:06:06,7055502,1823


In [47]:
initial_data.shape

(2575548, 14)

In [48]:
#give me only data with USEDTIME <300 i.e. 5 minutes
initial_data=initial_data[initial_data.USEDTIME<300]

In [49]:
initial_data.shape

(696855, 14)

In [50]:
initial_data=initial_data.groupby(['number','USER','EXITCODE'],as_index=False)[['JOBID']].count()

In [51]:
initial_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17488 entries, 0 to 17487
Data columns (total 4 columns):
number      17488 non-null object
USER        17488 non-null object
EXITCODE    17488 non-null object
JOBID       17488 non-null int64
dtypes: int64(1), object(3)
memory usage: 683.1+ KB


In [52]:
#this is the users who commited <500 jobs that were <5 minutes.
initial_data.head()

,number,USER,EXITCODE,JOBID
0,10000883,boyce,0:0,6
1,10007652,stacey,0:0,1
2,10007726,stacey,0:0,2
3,10007826,gerrit,0:0,50
4,10008050,gerrit,0:0,50


In [54]:
initial_data=initial_data[initial_data.JOBID>500]

In [55]:
#This is the number of times each person committed that criteria
initial_data.groupby('USER')[['number']].count()

,number
USER,
alf,3
alisson,1
archie,2
baylee,7
benson,19
boyce,31
brande,22
demarcus,7
hjalmar,2


In [ ]:
# keeping column with NODES =1
accre_cleaned=initial_data[accre_cleaned.NODES==1]

In [ ]:
# keeping column with value for EXITCODE=0:0
accre_cleaned=accre_cleaned[accre_cleaned.EXITCODE=='0:0']

In [ ]:
accre_cleaned.shape

In [ ]:
accre_cleaned.head()

In [ ]:
#dummy=accre_cleaned.head(10000)

In [ ]:
# keeping column with value for EXITCODE=0:0
#accre_cleaned=accre_cleaned[accre_cleaned.EXITCODE=='0:0']

In [ ]:
accre_cleaned.shape

In [ ]:
accre_cleaned.ACCOUNT.nunique()

In [ ]:
#calculating percent memory used
def percent_used(x,y):
    """calculating percentage from two columns"""
    return (y/x)*100

In [ ]:
#grouping by account and agrregating by total REQMEM , USEDMEM , USEDTIME
#accre_cleaned_sum=accre_cleaned.groupby('ACCOUNT').agg({'REQMEM':sum,'USEDMEM':sum,'USEDTIME':sum})


In [ ]:
#accre_cleaned_sum.head()

In [ ]:
#accre_cleaned_sum['TPUSED']=accre_cleaned_sum.apply(lambda row:percent_used(row['REQMEM'],row['USEDMEM']),axis=1)

In [ ]:
#accre_cleaned_sum=accre_cleaned_sum.sort_values('TPUSED', ascending=False)

In [ ]:
#accre_cleaned_sum

In [ ]:
# Average requested and used memory, sum total of used memory
#accre_cleaned_average= accre_cleaned.groupby('ACCOUNT')['REQMEM','USEDMEM','USEDTIME'].mean()

d={'REQMEM':'AVREQMEM','USEDMEM':'AVUSEDMEM','USEDTIME':'SUMUSEDMEM'}
accre_cleaned_average=accre_cleaned.groupby('ACCOUNT').agg({'REQMEM':'mean','USEDMEM':'mean','USEDTIME':'sum'}).rename(columns=d)

In [ ]:
accre_cleaned_average.head()

In [ ]:
accre_cleaned_average['AVPUSED']=accre_cleaned_average.apply(lambda row:percent_used(row['AVREQMEM'],row['AVUSEDMEM']),axis=1)

In [ ]:
accre_cleaned_average=accre_cleaned_average.sort_values('AVPUSED', ascending=False)

In [ ]:
accre_cleaned_average

In [ ]:
#Merge REQTIME and USEDTIME column

In [ ]:
#calculating weighted percent memory used
#def weighted_percent_used(x,y):
  #  return (x*y)/100